# ベイジアンネットワーク

## 流れ
1. データ取得
2. 要約統計量の算出
3. 犠打数,得点数,出塁率,打率,長打率の相関係数を算出
4. 犠打数と得点数の出塁率を固定した偏相関を算出
5. ベイジアンネットワークを作成

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import statsmodels as sm

# ベイジアンネットワーク用のライブラリは後で追加する

# データ取得

In [4]:
df = pd.read_csv('team_batting_14_23.csv')

# 先頭数行確認
df.head(10)

,チーム,打 率,試 合,打 席,打 数,得 点,安 打,二 塁 打,三 塁 打,本 塁 打,...,犠 打,犠 飛,四 球,故 意 四,死 球,三 振,併 殺 打,長 打 率,出 塁 率,年度
0,オリックス,0.250,143,5324,4782,508,1194,211,17,109,...,83,25,371,8,63,986,106,0.369,0.311,2023
1,ソフトバンク,0.248,143,5454,4786,536,1185,195,15,104,...,107,38,470,14,53,1053,88,0.360,0.319,2023
2,楽 天,0.244,143,5369,4667,513,1140,168,25,104,...,125,33,490,9,52,937,108,0.358,0.321,2023
3,ロッテ,0.239,143,5414,4744,505,1135,220,12,100,...,116,39,453,20,62,1011,79,0.354,0.311,2023
4,西 武,0.233,143,5222,4672,435,1088,188,21,90,...,90,28,387,9,45,1045,97,0.340,0.296,2023
5,日本ハム,0.231,143,5248,4688,464,1082,195,18,100,...,84,25,397,10,54,1111,78,0.344,0.297,2023
6,巨 人,0.252,143,5352,4826,523,1218,204,14,164,...,93,31,365,38,37,1111,94,0.402,0.308,2023
7,DeNA,0.247,143,5331,4783,520,1182,230,18,105,...,106,31,355,22,56,846,103,0.369,0.305,2023
8,阪 神,0.247,143,5479,4775,555,1180,179,34,84,...,106,47,494,35,57,1173,92,0.352,0.322,2023
9,広 島,0.246,143,5241,4728,493,1165,197,18,96,...,96,20,349,25,48,1032,115,0.357,0.304,2023


## 要約統計量算出

In [5]:
df.describe()

,打 率,試 合,打 席,打 数,得 点,安 打,二 塁 打,三 塁 打,本 塁 打,塁 打,...,犠 打,犠 飛,四 球,故 意 四,死 球,三 振,併 殺 打,長 打 率,出 塁 率,年度
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,...,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,0.251017,140.800000,5320.900000,4697.683333,554.050000,1179.850000,203.258333,21.300000,117.333333,1777.708333,...,102.633333,30.233333,440.933333,16.766667,49.250000,1020.041667,95.966667,0.378275,0.319875,2018.500000
std,0.010391,6.968838,290.064768,256.780912,72.770602,90.213253,21.671922,6.512561,30.172427,163.582652,...,24.661723,6.810439,62.001997,8.685672,11.766103,90.002651,12.618469,0.025603,0.013187,2.884324
min,0.229000,120.000000,4437.000000,3840.000000,390.000000,902.000000,152.000000,7.000000,62.000000,1365.000000,...,48.000000,15.000000,306.000000,2.000000,22.000000,811.000000,50.000000,0.326000,0.285000,2014.000000
25%,0.244750,143.000000,5305.500000,4704.750000,505.750000,1145.250000,188.750000,17.000000,94.750000,1676.750000,...,85.750000,26.000000,399.750000,10.000000,42.750000,956.000000,86.750000,0.359500,0.310750,2016.000000
50%,0.250500,143.000000,5386.000000,4775.000000,550.000000,1190.000000,205.000000,20.500000,111.500000,1773.000000,...,102.500000,30.000000,442.000000,15.500000,48.000000,1022.000000,95.000000,0.377500,0.320000,2018.500000
75%,0.257000,143.000000,5469.000000,4826.500000,594.500000,1232.250000,217.000000,26.000000,135.250000,1873.750000,...,115.000000,34.000000,484.000000,22.250000,56.250000,1084.500000,105.000000,0.396250,0.329000,2021.000000
max,0.280000,144.000000,5663.000000,5023.000000,792.000000,1401.000000,260.000000,39.000000,202.000000,2241.000000,...,178.000000,47.000000,599.000000,40.000000,87.000000,1234.000000,129.000000,0.454000,0.352000,2023.000000


## 興味のある変数を取り出して相関係数を算出

In [7]:
selected = df[['打  率', '得  点', '犠  打', '出 塁 率', '長 打 率']]
selected.corr()

,打 率,得 点,犠 打,出 塁 率,長 打 率
打 率,1.000000,0.733783,0.178846,0.775797,0.704033
得 点,0.733783,1.000000,0.054809,0.785257,0.826041
犠 打,0.178846,0.054809,1.000000,0.122307,-0.178494
出 塁 率,0.775797,0.785257,0.122307,1.000000,0.655911
長 打 率,0.704033,0.826041,-0.178494,0.655911,1.000000
